In [1]:
englishLetterFreq = {'E': 12.02, 'T': 9.1, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95, 'S': 6.28, 'H': 5.92, 'R': 6.02, 'D': 4.32, 'L': 3.98, 'C': 2.71, 'U': 2.88, 'M': 2.61, 'W': 2.09, 'F': 2.30, 'G': 2.03, 'Y': 2.11, 'P': 1.82, 'B': 1.49, 'V': 1.11, 'K': 0.69, 'J': 0.10, 'X': 0.17, 'Q': 0.11, 'Z': 0.07}

In [2]:
caracteres = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
transformador_a_numero = {c: caracteres.index(c) for c in caracteres}
transformador_a_letra = {caracteres.index(c):c for c in caracteres}

In [35]:
from collections import Counter

def most_likely_len(cypher):
  best = 1000000000000000000000000000000000000000000000000000000000000000000000000
  best_key = 0
  sorted_freq = sorted(englishLetterFreq.values(), reverse=True)
  for KEY_LEN in range(1, len(cypher)//50):
    sum = 0
    for pos in range(KEY_LEN):
      to_decypher = [cypher[x * KEY_LEN + pos] for x in range(len(cypher) // KEY_LEN)]
      count = Counter(to_decypher)
      sorted_count = sorted(count.values(), reverse=True)
      for i in range(len(sorted_freq)):
        if i < len(sorted_count):
          sum += abs((sorted_freq[i] - sorted_count[i]/(len(cypher) // KEY_LEN)))
        else:
          sum += sorted_freq[i]
    sum /= KEY_LEN
    if sum < best:
      best_key = KEY_LEN
      best = sum
  return best_key

In [40]:
def most_likely_key_length(ciphertext,
    frequencies,
    distance):
    max_length = len(ciphertext)//50
    real_key_length = 1
    
    alphabet = sorted(frequencies.keys())
    freq_sort = sorted(frequencies.values())
    max_diff = 10000000000000000000000000000000000000000000000000000000000000000000000000
    
    for length in range(1,max_length+1):
        compare_array = []
        sum = 0
        for pos in range(length):
            compare_dict = {k:0 for k in frequencies.keys()}
            pos_len = len(ciphertext)//length
            to_decrypt = [ciphertext[i*length + pos] for i in range(pos_len)]

            for letter in to_decrypt:
                compare_dict[letter] += 1

            sorted_count = sorted(compare_dict.values())
            for i in range(len(sorted_count)):
                if i < len(sorted_count):
                    sum += abs((freq_sort[i] - sorted_count[i]/pos_len))
                else:
                    sum += freq_sort[i]
        sum /= length
        if sum < max_diff:
            real_key_length = length
            max_diff = sum
    return real_key_length

In [3]:
def encrypt_otp(texto,key):
    cypher_text = ""
    cypher_number = []
    new_key = ""
    while len(new_key) < len(texto):
        new_key += key
    
    key = new_key[0:len(texto)]



    for t,k in zip(texto,key):
        cypher_number.append((transformador_a_numero[t] + transformador_a_numero[k])%len(englishLetterFreq))
        cypher_text += transformador_a_letra[cypher_number[-1]]

    print(cypher_text)

In [4]:
def decrypt_otp(cipher,key):
    key = "".join(key)
    text = ""
    number = []
    new_key = ""
    while len(new_key) < len(cipher):
        new_key += key
    
    key = new_key[0:len(cipher)]
    
    for t,k in zip(cipher,key):
        number.append((transformador_a_numero[t] - transformador_a_numero[k])%len(englishLetterFreq))
        text += transformador_a_letra[number[-1]]

    return text


In [5]:
import itertools
import math


In [6]:
def most_likely_char(pos, ciphertext,key_len,frequencies,distance):
    to_decrypt = [ciphertext[i*key_len + pos] for i in range(len(ciphertext)//key_len)]
    alphabet = sorted(frequencies.keys())
    best_char = alphabet[0]
    best_distance = len(alphabet)

    for candidate in alphabet:
        string = ''
        for c in to_decrypt:
            n_c = alphabet.index(c)
            n_candidate = alphabet.index(candidate)
            n = (n_c - n_candidate) % len(alphabet)
            string += alphabet[n]
        dif = distance(string,frequencies)
        if best_distance > dif:
            best_char = candidate
            best_distance = dif
    return best_char
            
            
    

In [41]:
def break_rp(
    ciphertext,
    frequencies,
    distance,
) -> str:

    key_len = most_likely_key_length(
        ciphertext,
        frequencies,
        distance,
    )
    key = ''
    print(key_len)
    for i in range(key_len):
        key += most_likely_char(i, ciphertext, key_len, frequencies, distance)

    return decrypt_otp(ciphertext, key)


In [9]:
def abs_distance( string , frequencies) -> float :
    return sum([
    abs((frequencies[c]/100) - (string.count(c) / len(string)) )
    for c in frequencies
    ])


In [38]:
cipher = "TIEFNJTPDLIYGEOXOGGCEBTMRJTLIOAYDOOCTIECNJRPLBNOOSUVITIYWFSEESNPUSOAEJTNONPCITEDTIETSMAYDPFRRFAEBSIEAJNPNHLLNESNOULLNEAYDXAWETAYDUHPNPREHFRYOOEDIYTSOGTSEJSWAODZFJRPLBNONPREHFRYISEWAODEOHEEHFRHIUHXAOYDMBLWESIDLBNOSUHPMBIYLBNOASELSMIPBFTHEFNWAUIEUEEDNBNONUHPSIEELBNOITLLNESCEBCSTPNPASLJNBNOLPNRIUUOETWEOFTSESOJAMGCEFNHIDHZBTECVBTZRZNPASLZNEOYITTSEEEQIOIYGQOTNUOQTIEARJMPMFRTDJAYTIEFNJTPDLIYGEOXHBSLTPTLLBRPAPFLPQRZXJMLTFLJKNTSEVKWIFSMEUWPEOTSEOOCTIAELBNEIDAYDUHPNPREHTELAODNONEDWJTSIOKXMJOQTIEYOSTSWFSECPADTPFQRBNNEGRZMXHTCIIEITSPPBRLTFDMYUHPEOGWITHNHBNYEMIESIACETAVMJNEESNLTJOYAMLLNEBZUODLRZWTTITSESEAUCLTCPFTRFLLNETSEDHLNOEWTVNYEMBZRFDMEOELTITSEFNRLJSSCIAYNFLYOXLTNLSEHFUVWJTSFSAYCFTSEQHJSJCLLHEZGSAAHZOQTIEFKWACIFSRRFAELZEYGMAYDDOYSJSESPFXOTTWYMOHLBNOTFRCAJNHIUHFPMAYDPRXOVNEAJNZUTTPRSATNPNWYGOFNENZRUHHETTZFUHPTFEDEYEWIOEEHFUALBNOASELSJNNLVDPTIEWALEOITTCIDTEHFPPNOIYETNZRUHJOSKXOPRDEYMZOSAYDEACTNOZRUHPLPWWAODLRFADASEEYQINAMLJTSAGESSPDCYCAOGPSPFWOXHTLMSQRFQFEOTWYDOXPPSPDPFNHBLVAODQLBTALBIYSTCZTMAYDJSEHFMZSUMZUOTLIOOFSDOFNURJIOTSEVKLNEIESQHJSJCLLHEZGSAAHZIDDJSEIOGFITHPDCYEHFHTGILLNEBZUODLRZFLUMTHHJCSTSAGESSPSUHPSDOETJSSMBIYLBNOFSOXHFLPNTBFRHHEOTTZNFHLVFNEHFFLUMTWIOEDEQACAUEDTIEEWPDTSUIYCUIGEMYOIGFPRFNERFGTOOSZFUHPHJGSLBNOSUOEHFNZRUHLNEWPSUAYDUHPLPWWAODDTPTSETOFTIAYDFADTUHPHJGSLBNOSBRPPSEOONIYAOTWYNOFNUATNPUDCPNEAJNTNHTSENAUOSIEYPFDCPTWAODDMPUYTBIYOVSWAODDCBPPWIIWEUHPLPWWAODDCPNEAJNQLBTEESLLNEEDPFCTAMLJADRZSTTSEDEYTSAWLPWWAODDWJTSUQLLNEAYDNOFNUATNPUDTFRCAJNWODAEEEAETIEDOVTSESNFPMAYDTWLLFSTSNODTMYXOVNEAJNZUTTSOVGSSPUEHXAWETIDLFSDMPUYTBIYOVSEHBNYOSTSAODXIEWLLFSYOSTSESNTRFLLNECZNTIDTTOQMPSELZHTLMYWAODDCBPPAODTTTGPOHRLPIYTNDLFDFSEHFMZUSNPMPUYTBIYSBSHEMLLSMOFGINPAHHLTTQFASEVIMOXEURPSTQXIUHPLBRRETTMOEYZFXAEESIYTIEFKUHPOWECAMLREPMZRQHZLPGJOGTSEVKHATSSAQEOBZANONBTNBTTOOOQFPRNETIYCMUOIOGEEDTZNJCDAODNLJMLTFCSAOGPIOPLRUINUMACGMANIBTTOOIYNPREHFRYAODHETTPROACEBSEHFTLLMEDTNOFNUATNJNEHFUVAODMRJTTSIIDLFSTSCEYNFVTSJNEHFGCANPTAOMZUOTLIOSDCPTWAODEHFLZNHEDTSIGESIDTIECIWECSFVPROWSIDHQLPWDFSOXWBLPSJNEOFNRLBNOTIEWASGPSULLKFBJSVRQADELRFATSMOFGINPAHHTNOOCTIECNJRPLBNOTIOFGISNOULLNESWODHYETSSATTSEMACGFSEVPLFMF"

In [42]:
print(break_rp (
    cipher,
    englishLetterFreq,
    abs_distance,
))

36
THEUNITEDKINGDOMOFGREATBRITAINANDNORTHERNIRELANDORUKISINWESTERNEUROPEITCOMPRISESTHEISLANDOFGREATBRITAINENGLANDSCOTLANDANDWALESANDTHENORTHERNONESIXTHOFTHEISLANDOFIRELANDNORTHERNIRELANDTOGETHERWITHMANYSMALLERISLANDSTHEMAINLANDAREASLIEBETWEENLATITUDESNANDNTHESHETLANDISLANDSREACHTONEARLYNANDLONGITUDESWTOETHEROYALGREENWICHOBSERVATORYNEARLONDONISTHEDEFININGPOINTOFTHEPRIMEMERIDIANTHEUNITEDKINGDOMHASATOTALAREAOFAPPROXIMATELYKMTHEUKLIESBETWEENTHENORTHATLANTICANDTHENORTHSEAANDCOMESWITHINKMMIOFTHENORTHWESTCOASTOFFRANCEFROMWHICHITISSEPARATEDBYTHEENGLISHCHANNELITSHARESAKMINTERNATIONALLANDBOUNDARYWITHTHEREPUBLICOFIRELANDTHECHANNELTUNNELBOREDBENEATHTHEENGLISHCHANNELNOWLINKSTHEUKWITHFRANCETHEPHYSICALGEOGRAPHYOFTHEUKVARIESGREATLYENGLANDCONSISTSOFMOSTLYLOWLANDTERRAINWITHUPLANDORMOUNTAINOUSTERRAINONLYFOUNDNORTHWESTOFTHETEESEXELINETHEUPLANDAREASINCLUDETHELAKEDISTRICTTHEPENNINESNORTHYORKMOORSEXMOORANDDARTMOORTHELOWLANDAREASARETYPICALLYTRAVERSEDBYRANGESOFLOWHILLSFREQUENTLYCOMPOSEDOFCHALKANDFLATPLAINSSCOT

In [11]:
print(abs_distance(new_s,englishLetterFreq))

NameError: name 'new_s' is not defined